In [2]:
# Import statements
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xesmf as xe
# Load dictionary which containes all 19 year trends
TrendDictionary = pd.read_pickle(r'/home/disk/pna2/aodhan/ts_trends.pickle')

In [15]:
models_for_training_count = 0 # Count of models with at least 10 ensembles
for model in TrendDictionary.keys():
    if (len(TrendDictionary[model].keys())) >= 11:
        models_for_training_count +=1
        # CMIP6 models must be regridded, below we define input and output grids
        InputGrid = TrendDictionary[model]['grid']
        OutputGrid = {"lon": np.arange(1.25, 358.751, 2.5), "lat": np.arange(-88.75, 88.751, 2.5)}
        regridder = xe.Regridder(InputGrid, OutputGrid, "bilinear")
        
        EnsembleTrendsArray = []
        for ensemble_name in TrendDictionary[model].keys():
            if ensemble_name == 'grid':
                continue
            ensemble_member = TrendDictionary[model][ensemble_name]
            
            # return the timeperiods and trend maps in ensemble_member dictionary
            result = ensemble_member.items()
            data = np.array(list(result))
            timeperiods = data[:,0] # not sure if I will need this
            trend_maps = data[:,1] # these are the indivisual 19 year trend maps
            
            # Regrid trend map to 2.5 x 2.5 grid
            trend_maps_2p5x2p5 = [regridder(trend_maps[i]) for i in range(0, len(trend_maps))]

            # Return all ensemble trends 
            EnsembleTrendsArray.append(trend_maps_2p5x2p5)
        EnsembleTrendsArray = np.array(EnsembleTrendsArray)
        ForcedTrend = np.nanmean(EnsembleTrendsArray, axis=0)
        NaturalTrendsArray = EnsembleTrendsArray - ForcedTrend
print(models_for_training_count)

/home/disk/p/aodhan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


20
